In [1]:
import json
import numpy as np
from IPython.display import display, HTML
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import os

In [2]:
planner_names = [
    "MTGTaskPlanner",
    "MTGTaskPlannerWithGMM",
#     "DynamicMTGTaskPlannerWithGMM",
#     "DynamicMTGTaskPlannerWithGMM_V2",
    "DynamicMTGTaskPlannerWithGMM_25.0",
#     "DynamicMTGTaskPlannerWithGMM_50.0",
#     "DynamicMTGTaskPlannerWithGMM_75.0",
    # "DynamicMTGTaskPlannerWithGMM_100.0",
    "MTGTaskPlannerWithAtlas",
#     "DynamicMTGPlannerWithAtlas",
#     "DynamicMTGPlannerWithAtlas_V2",
#     "DynamicMTGPlannerWithAtlas_25.0",
    "DynamicMTGPlannerWithAtlas_50.0",
#     "DynamicMTGPlannerWithAtlas_75.0",
    # "DynamicMTGTaskPlannerWithAtlas_100.0",

]


In [3]:
def read_json(file_path):
    """
    Reads a json file
    """
    result_data = []
    # Open the file and load JSON data
    with open(file_path, 'r') as json_file:
        for line in json_file:
            try:
                result_data.append(json.loads(line))
            except:
                continue
    return result_data
    

In [20]:
filepaths = [
            "../check/self_generated/maze/result_gmm.json", 
            "../check/self_generated/open_wardrobe/result_gmm.json", 
            "../check/self_generated/open_bottle/result_gmm.json", 
            "../check/self_generated/open_drawer/result_gmm.json", 
            "../check/self_generated/shelf/result_gmm.json", 
            "../check/self_generated/pour_water/result_gmm.json", 
            ]

problems = [
    "maze",
    "open_wardrobe",
    "open_bottle",
    "open_drawer",
    "shelf",
    "pour_water"
]

all_dfs = pd.DataFrame()

for problem in problems:
    file_path = "../check/self_generated/%s/result_gmm_2.json"%problem
    print(problem)
    if not os.path.exists(file_path):
        file_path = "../check/self_generated/%s/result_gmm.json"%problem
        if not os.path.exists(file_path):
            continue
        
    
    result_data = read_json(file_path)
    planner_stats = {
                        planner_name: {
                            'num' : 0, 
                            'num_success' : [], 
                            'motion_planning_time' : [],
                            'task_planning_time' : [],
                            'updating_time' : [],
                            'solution_length' : [],
                            'num_attempts' : [],
                            'total_planning_time' : [],
                            'set_start_and_goal_time' : [],
                        } 

                     for planner_name in planner_names
                    }

    for d in result_data:
        if d['planner_name'] in planner_stats:
            dic = planner_stats[d['planner_name']]
            dic['num'] += 1
            dic['num_success'].append(d['success'] == "true")
            if d['success'] == "true":
                dic['motion_planning_time'].append(d['motion_planning_time'])
                dic['task_planning_time'].append(d['task_planning_time'])
                dic['total_planning_time'].append(d['total_planning_time'])
                dic['updating_time'].append(d['updating_time'])
                dic['solution_length'].append(d['solution_length'])
                dic['num_attempts'].append(d['num_attempts'])
                dic['set_start_and_goal_time'].append(d['set_start_and_goal_time'])

    output_dict = []
    for planner_name in planner_names:
        stats = planner_stats[planner_name]

        current_planner_eval_dict = {}
        current_planner_eval_dict["Planner Name"] = planner_name.split("_")[0]
        current_planner_eval_dict["Success Rate"] = np.mean(stats['num_success']) * 100.0
        current_planner_eval_dict["Total Planning Time"] = np.mean(np.array(stats['total_planning_time']) )
        current_planner_eval_dict["Mean number of attempts"] = np.mean((np.array(stats['num_attempts'])) )
        current_planner_eval_dict["Mean Task Planning Time"] = np.mean(np.array(stats['task_planning_time']) / (np.array(stats['num_attempts'])) )
        current_planner_eval_dict["Mean Update time"] = np.mean(np.array(stats['updating_time']) / (np.array(stats['num_attempts'])) )
        current_planner_eval_dict["Mean solution length"] = np.mean(np.array(stats['solution_length']))# / (np.array(stats['num_attempts'])) )
#         current_planner_eval_dict["Total Number of Tasks"] = len(stats['num_success'])
#         current_planner_eval_dict["Mean Motion Planning Time"] = np.mean(np.array(stats['motion_planning_time']))# / (np.array(stats['num_attempts'])) )
#         current_planner_eval_dict["Set Start and Goal Time"] = np.mean(np.array(stats['set_start_and_goal_time']) )
        output_dict.append(current_planner_eval_dict)
    df = pd.DataFrame(output_dict).set_index("Planner Name")[
        [
            "Success Rate",
            "Total Planning Time",
            "Mean number of attempts",
            "Mean Task Planning Time",
            "Mean Update time",
            "Mean solution length",
        ]
    ]
    all_dfs = pd.concat([all_dfs,df], axis = 0 )
#     df.style.apply(bold_style, axis=None).to_latex()

    display(df)
    print(df.to_latex())
# all_dfs

maze


,Success Rate,Total Planning Time,Mean number of attempts,Mean Task Planning Time,Mean Update time,Mean solution length
Planner Name,,,,,,
MTGTaskPlanner,6.000,40.488,24.000,0.028,0.047,22.439
MTGTaskPlannerWithGMM,72.000,17.762,7.222,0.404,0.253,26.656
DynamicMTGTaskPlannerWithGMM,72.000,13.920,6.056,0.179,0.051,25.822
MTGTaskPlannerWithAtlas,76.000,9.902,4.289,0.353,0.195,22.443
DynamicMTGPlannerWithAtlas,78.000,8.041,3.821,0.080,0.047,24.884


\begin{tabular}{lrrrrrr}
\toprule
{} &  Success Rate &  Total Planning Time &  Mean number of attempts &  Mean Task Planning Time &  Mean Update time &  Mean solution length \\
Planner Name                 &               &                      &                          &                          &                   &                       \\
\midrule
MTGTaskPlanner               &         6.000 &               40.488 &                   24.000 &                    0.028 &             0.047 &                22.439 \\
MTGTaskPlannerWithGMM        &        72.000 &               17.762 &                    7.222 &                    0.404 &             0.253 &                26.656 \\
DynamicMTGTaskPlannerWithGMM &        72.000 &               13.920 &                    6.056 &                    0.179 &             0.051 &                25.822 \\
MTGTaskPlannerWithAtlas      &        76.000 &                9.902 &                    4.289 &                    0.353 &             0.

,Success Rate,Total Planning Time,Mean number of attempts,Mean Task Planning Time,Mean Update time,Mean solution length
Planner Name,,,,,,
MTGTaskPlanner,78.000,91.577,14.128,0.034,0.070,32.304
MTGTaskPlannerWithGMM,86.000,54.896,8.140,0.328,0.131,24.743
DynamicMTGTaskPlannerWithGMM,88.000,42.480,8.636,0.074,0.024,24.432
MTGTaskPlannerWithAtlas,94.000,42.810,6.830,0.280,0.100,20.740
DynamicMTGPlannerWithAtlas,90.000,40.511,7.311,0.029,0.021,21.504


\begin{tabular}{lrrrrrr}
\toprule
{} &  Success Rate &  Total Planning Time &  Mean number of attempts &  Mean Task Planning Time &  Mean Update time &  Mean solution length \\
Planner Name                 &               &                      &                          &                          &                   &                       \\
\midrule
MTGTaskPlanner               &        78.000 &               91.577 &                   14.128 &                    0.034 &             0.070 &                32.304 \\
MTGTaskPlannerWithGMM        &        86.000 &               54.896 &                    8.140 &                    0.328 &             0.131 &                24.743 \\
DynamicMTGTaskPlannerWithGMM &        88.000 &               42.480 &                    8.636 &                    0.074 &             0.024 &                24.432 \\
MTGTaskPlannerWithAtlas      &        94.000 &               42.810 &                    6.830 &                    0.280 &             0.

...

\begin{tabular}{lrrrrrr}
\toprule
{} &  Success Rate &  Total Planning Time &  Mean number of attempts &  Mean Task Planning Time &  Mean Update time &  Mean solution length \\
Planner Name                 &               &                      &                          &                          &                   &                       \\
\midrule
MTGTaskPlanner               &        76.000 &               87.573 &                   18.789 &                    0.031 &             0.043 &                37.487 \\
MTGTaskPlannerWithGMM        &        98.000 &               19.664 &                    3.939 &                    0.188 &             0.139 &                28.587 \\
DynamicMTGTaskPlannerWithGMM &        98.000 &               11.761 &                    3.571 &                    0.024 &             0.027 &                27.199 \\
MTGTaskPlannerWithAtlas      &       100.000 &               11.845 &                    2.560 &                    0.139 &             0.

,Success Rate,Total Planning Time,Mean number of attempts,Mean Task Planning Time,Mean Update time,Mean solution length
Planner Name,,,,,,
MTGTaskPlanner,29.412,69.902,9.800,0.552,0.154,40.336
MTGTaskPlannerWithGMM,74.000,37.134,3.784,0.875,0.368,41.557
DynamicMTGTaskPlannerWithGMM,74.000,41.405,4.838,0.176,0.136,37.943
MTGTaskPlannerWithAtlas,98.000,26.564,2.837,0.887,0.332,30.966
DynamicMTGPlannerWithAtlas,98.000,25.827,2.898,0.151,0.132,32.844


\begin{tabular}{lrrrrrr}
\toprule
{} &  Success Rate &  Total Planning Time &  Mean number of attempts &  Mean Task Planning Time &  Mean Update time &  Mean solution length \\
Planner Name                 &               &                      &                          &                          &                   &                       \\
\midrule
MTGTaskPlanner               &        29.412 &               69.902 &                    9.800 &                    0.552 &             0.154 &                40.336 \\
MTGTaskPlannerWithGMM        &        74.000 &               37.134 &                    3.784 &                    0.875 &             0.368 &                41.557 \\
DynamicMTGTaskPlannerWithGMM &        74.000 &               41.405 &                    4.838 &                    0.176 &             0.136 &                37.943 \\
MTGTaskPlannerWithAtlas      &        98.000 &               26.564 &                    2.837 &                    0.887 &             0.

,Success Rate,Total Planning Time,Mean number of attempts,Mean Task Planning Time,Mean Update time,Mean solution length
Planner Name,,,,,,
MTGTaskPlanner,92.000,83.302,7.761,2.455,0.637,43.789
MTGTaskPlannerWithGMM,100.000,15.606,1.560,2.806,0.698,32.127
DynamicMTGTaskPlannerWithGMM,100.000,6.399,1.420,0.002,0.257,31.267
MTGTaskPlannerWithAtlas,100.000,12.804,1.420,2.532,0.681,26.081
DynamicMTGPlannerWithAtlas,100.000,7.781,1.420,0.003,0.261,27.835


\begin{tabular}{lrrrrrr}
\toprule
{} &  Success Rate &  Total Planning Time &  Mean number of attempts &  Mean Task Planning Time &  Mean Update time &  Mean solution length \\
Planner Name                 &               &                      &                          &                          &                   &                       \\
\midrule
MTGTaskPlanner               &        92.000 &               83.302 &                    7.761 &                    2.455 &             0.637 &                43.789 \\
MTGTaskPlannerWithGMM        &       100.000 &               15.606 &                    1.560 &                    2.806 &             0.698 &                32.127 \\
DynamicMTGTaskPlannerWithGMM &       100.000 &                6.399 &                    1.420 &                    0.002 &             0.257 &                31.267 \\
MTGTaskPlannerWithAtlas      &       100.000 &               12.804 &                    1.420 &                    2.532 &             0.

In [19]:
def bold_style(val):
    f = "font-weight: bold" 
    #condition
    print(val)
    mask_max = val.eq(val == 0)
    max_cols = ["Success Rate"]
    min_cols = ["Total Planning Time", "Mean number of attempts", "Mean Task Planning Time", "Mean Update time", "Mean solution length"]
    mask_max[min_cols] = val[min_cols].eq(val[min_cols].min())
    mask_max[max_cols] = val[max_cols].eq(val[max_cols].max())
    # DataFrame of styles
    df1 = pd.DataFrame('', index=val.index, columns=val.columns)
    # set columns by condition
    df1 = df1.mask(mask_max, f)
    return df1

dfx = df.style.map(bold_style)
display(dfx)

AttributeError: 'Styler' object has no attribute 'map'

In [7]:
pd.__version__

'1.1.5'

In [8]:
!pip3 install pandas==1.3.0

ERROR: Could not find a version that satisfies the requirement pandas==1.3.0 (from versions: 0.1, 0.2, 0.3.0, 0.4.0, 0.4.1, 0.4.2, 0.4.3, 0.5.0, 0.6.0, 0.6.1, 0.7.0, 0.7.1, 0.7.2, 0.7.3, 0.8.0, 0.8.1, 0.9.0, 0.9.1, 0.10.0, 0.10.1, 0.11.0, 0.12.0, 0.13.0, 0.13.1, 0.14.0, 0.14.1, 0.15.0, 0.15.1, 0.15.2, 0.16.0, 0.16.1, 0.16.2, 0.17.0, 0.17.1, 0.18.0, 0.18.1, 0.19.0, 0.19.1, 0.19.2, 0.20.0, 0.20.1, 0.20.2, 0.20.3, 0.21.0, 0.21.1, 0.22.0, 0.23.0, 0.23.1, 0.23.2, 0.23.3, 0.23.4, 0.24.0, 0.24.1, 0.24.2, 0.25.0, 0.25.1, 0.25.2, 0.25.3, 1.0.0, 1.0.1, 1.0.2, 1.0.3, 1.0.4, 1.0.5, 1.1.0, 1.1.1, 1.1.2, 1.1.3, 1.1.4, 1.1.5)
ERROR: No matching distribution found for pandas==1.3.0
